In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from neo4j import GraphDatabase
from scipy import linalg
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

In [2]:
myfilename = "word_pairs.txt"

with open(myfilename) as f:
    df = json.load(f)

Create unique node list for graph.

In [8]:
word_pairs = set([item for sublist in [item[0] for item in df] for item in sublist])

Generate a list of all the nodes.

In [9]:
nodes = []
for item in word_pairs:
    nodes.append({"Word": item})
print("Total number of nodes:", len(nodes))

Total number of nodes: 801


Generate a list of all the relationships between nodes with probability as weight.

In [11]:
relationships = []
for item in df:
    relationships.append({"from":item[0][0],"to":item[0][1], "properties":{"probability":item[1]}})
            
print("Total number of relationships:", len(relationships))

Total number of relationships: 1000


Connect to neo4j database.

In [12]:
url = "bolt://localhost:7687"
username = "neo4j"
password = "scribbles"

# Connect to the neo4j database server
graphDB_Driver  = GraphDatabase.driver(url, auth=(username, password))
graphDB_Driver.open

<bound method BoltDriver.open of <class 'neo4j._sync.driver.BoltDriver'>>

Run the query to create all nodes in neo4j.

In [13]:
graphDB_Driver.session().run(
    query = 'UNWIND $props AS map CREATE (n:Word) SET n = map',
    parameters = {'props': nodes}
)

Run the query to create all relationships in neo4j.

In [14]:
graphDB_Driver.session().run(
    query = 'UNWIND $batch as row \
    MATCH (from:Word {Word: row.from}) \
    MATCH (to:Word {Word: row.to}) \
    CREATE (from)-[rel:KNOWS]->(to) \
    SET rel += row.properties', 
    parameters = {'batch': relationships}
)

Delete all nodes and relationships.

In [15]:
graphDB_Driver.session().run(
    query = 'match (n) detach delete n', 
    parameters = {'batch': relationships}
)